In [1]:
import numpy as np
import pandas as pd
from scipy.special import softmax
import csv
import urllib.request
import ssl
import certifi
import os

In [2]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

#tokenizer = AutoTokenizer.from_pretrained(MODEL)
#model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(save_directory='../models/' + MODEL)
#tokenizer.save_pretrained(save_directory='../models/' + MODEL)

tokenizer = AutoTokenizer.from_pretrained('../models/' + MODEL, local_files_only=True)
model = TFAutoModelForSequenceClassification.from_pretrained('../models/' + MODEL, local_files_only=True)


2022-02-25 13:36:45.458086: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at ../models/cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [3]:
# download label mapping
labels=[]
task='sentiment'
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link, context=ssl.create_default_context(cafile=certifi.where())) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

In [4]:
# # TF Examples
#Yeah exactly, dxy is pumping and every thing is still green.. SPX and DJI and BTC.. struggling yeah, but not dumping though, given that bulls and bears are calling for the 38k.. 💁🏻‍♂️ So where do you see us go from here? In the next weeks and until the end of year?
#"I think bitcoin is really on the verge of getting broad acceptance by conventional finance people"
#neu1 = "WHY CHINA’S RENEWED HARSH CRACKDOWN ON CRYPTO ISN’T STIFLING BITCOIN — AT LEAST NOT YET #bitcoin #btc"
#text1 = "If Bitcoin was able to capture a $1T market cap, it's not even impossible at this point. It might take time. But there is a chance."
positive = "If SEC approves the one of the ETF’s , this will allow people and instutitional investors to buy bitcoin easier."
#neg1 = "yesterday $btc suffered the largest percentage drop in a single day since june 21st📉, falling by almost $6,000! are you buying the dip or do you think we'll see more downside? bet on the price of (⬆️ or ⬇️) with up to x1000 leverage👉:"
#neg2 = "problems continue to plague el salvador's bitcoin rollout"
def predict(text):
    encoded_input = tokenizer(text, return_tensors='tf')
    output = model(encoded_input)
    scores = output[0][0].numpy()
    scores = softmax(scores)
    print(scores)
    print(labels[0])
    print(labels[1])
    print(labels[2])
    """ranking = np.argsort(scores)
    ranking = ranking[::-1]
    for i in range(scores.shape[0]):
        l = labels[ranking[i]]
        s = scores[ranking[i]]
        print(f"{i+1}) {l} {np.round(float(s), 4)}")"""
predict(positive)

[0.00341765 0.31853524 0.67804724]
negative
neutral
positive


In [5]:
#load preprocessed dataset
data = pd.read_csv('../data/processed/bitcoin_tweets_result_processed_20211001-20220130.csv', index_col=0)


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
print(data.shape)
data.head()
data.info()

(8919051, 13)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8919051 entries, 0 to 9168746
Data columns (total 13 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   id                 int64 
 1   date               object
 2   text               object
 3   hashtags           object
 4   replyCount         int64 
 5   retweetCount       int64 
 6   likeCount          int64 
 7   userName           object
 8   userFollowerCount  int64 
 9   userFavCount       int64 
 10  userFriendCount    int64 
 11  dateTime           object
 12  textLength         int64 
dtypes: int64(8), object(5)
memory usage: 952.7+ MB


In [16]:
#select a slice of the dataset for prediction, based on dateTime
#data = data.drop(['Unnamed: 0'])

sData = data[(data['dateTime'] >= '2022-01-10') & (data['dateTime'] < '2022-01-30')]

sData = sData.dropna(subset=['text'])

print(sData[sData['text'].map(type)== str].shape) 

print('the number of selected texts:', sData.shape[0])

print('estimated completion time:', sData.shape[0] * 0.16 / 60 / 60, 'hours')

(1618690, 13)
the number of selected texts: 1618690
estimated completion time: 71.94177777777777 hours


In [20]:
#apply prediction on texts
def predict(text):
    try:
        encoded_input = tokenizer(text, return_tensors='tf')
        output = model(encoded_input)
        scores = output[0][0].numpy()
        scores = softmax(scores)
        negative = np.round(float(scores[0]), 4)
        neutral = np.round(float(scores[1]), 4)
        positive = np.round(float(scores[2]), 4)
        ranking = np.argsort(scores)
        ranking = ranking[::-1]
        prediction = labels[ranking[0]]
        #score = scores[ranking[0]]
        #confidence = np.round(float(score), 4)
        return pd.Series([prediction, negative, neutral, positive])
    except:
        print('error:', text)
        return pd.Series(['unknown', 0,0,0])

#btcdata['sentiment'] = btcdata['text'].apply(lambda x: x.lower())
sData[['prediction','negative', 'neutral', 'positive']] = sData['text'].apply(lambda x: predict(x))

error: all countries can now invest 🇺🇳🇦🇫🇦🇽🇦🇱🇦🇮🇦🇴🇦🇩🇦🇸🇩🇿🇦🇶🇦🇬🇦🇷🇦🇲🇦🇼🇧🇭🇧🇸🇦🇿🇦🇹🇦🇺🇧🇩🇧🇧🇧🇾🇧🇪🇧🇿🇧🇦🇧🇴🇧🇹🇧🇲🇧🇯🇧🇼🇧🇷🇮🇴🇻🇬🇨🇲🇧🇳🇰🇭🇧🇮🇧🇫🇧🇬🇨🇦🇮🇨🇨🇻🇧🇶🇰🇾🇨🇽🇨🇳🇨🇱🇹🇩🇨🇫🇨🇨🇨🇴🇰🇲🇨🇬🇨🇩🇨🇺🇭🇷🇨🇮🇨🇷🇨🇰🇨🇼🇨🇾🇨🇿🇩🇰🇩🇯🇸🇻🇪🇬🇪🇨🇩🇴🇩🇲🇬🇶🇪🇷🇪🇪🇸🇿🇪🇹🇫🇮🇫🇯🇫🇴🇫🇰🇪🇺🇫🇷🇬🇫🇵🇫🇹🇫🇬🇦🇬🇮🇬🇭🇩🇪🇬🇪🇬🇲🇬🇷🇬🇱🇬🇩🇬🇵🇬🇺🇬🇾🇬🇼🇬🇳🇬🇬🇬🇹🇭🇹🇭🇳🇭🇰🇭🇺🇮🇸🇮🇪🇮🇶🇮🇷🇮🇩🇮🇳🇮🇲🇮🇱🇮🇹🇯🇲🇯🇵🇰🇪🇰🇿🇯🇴🇯🇪
error: all countries can now invest 🇺🇳🇦🇫🇦🇽🇦🇱🇦🇮🇦🇴🇦🇩🇦🇸🇩🇿🇦🇶🇦🇬🇦🇷🇦🇲🇦🇼🇧🇭🇧🇸🇦🇿🇦🇹🇦🇺🇧🇩🇧🇧🇧🇾🇧🇪🇧🇿🇧🇦🇧🇴🇧🇹🇧🇲🇧🇯🇧🇼🇧🇷🇮🇴🇻🇬🇨🇲🇧🇳🇰🇭🇧🇮🇧🇫🇧🇬🇨🇦🇮🇨🇨🇻🇧🇶🇰🇾🇨🇽🇨🇳🇨🇱🇹🇩🇨🇫🇨🇨🇨🇴🇰🇲🇨🇬🇨🇩🇨🇺🇭🇷🇨🇮🇨🇷🇨🇰🇨🇼🇨🇾🇨🇿🇩🇰🇩🇯🇸🇻🇪🇬🇪🇨🇩🇴🇩🇲🇬🇶🇪🇷🇪🇪🇸🇿🇪🇹🇫🇮🇫🇯🇫🇴🇫🇰🇪🇺🇫🇷🇬🇫🇵🇫🇹🇫🇬🇦🇬🇮🇬🇭🇩🇪🇬🇪🇬🇲🇬🇷🇬🇱🇬🇩🇬🇵🇬🇺🇬🇾🇬🇼🇬🇳🇬🇬🇬🇹🇭🇹🇭🇳🇭🇰🇭🇺🇮🇸🇮🇪🇮🇶🇮🇷🇮🇩🇮🇳🇮🇲🇮🇱🇮🇹🇯🇲🇯🇵🇰🇪🇰🇿🇯🇴🇯🇪


In [19]:
#sData.groupby('date')['id'].count()
sData.head()


,id,date,text,hashtags,replyCount,retweetCount,likeCount,userName,userFollowerCount,userFavCount,userFriendCount,dateTime,textLength
0,1487576284834287618,2022-01-29 23:59:58+00:00,where are all my north carolina bitcoin lovers...,NaN,1,2,15,TheCoinDad,4988,7414,507,2022-01-29 23:59:58+00:00,75
1,1487576278139998208,2022-01-29 23:59:57+00:00,closed at break even. waiting for btc direction.,NaN,1,0,0,thouughtless,2,11,37,2022-01-29 23:59:57+00:00,48
2,1487576273647902720,2022-01-29 23:59:56+00:00,i'm not a fan of elon but putting your life sa...,NaN,2,0,0,AlienParty,213,5408,671,2022-01-29 23:59:56+00:00,91
3,1487576272574324738,2022-01-29 23:59:55+00:00,if chlamydia could talk…,NaN,84,65,598,maxkeiser,449479,18817,1724,2022-01-29 23:59:55+00:00,24
4,1487576268304367618,2022-01-29 23:59:54+00:00,btc outperformed eth from 2009- 2015. but eth ...,NaN,2,0,1,Spicytaco34,11,733,353,2022-01-29 23:59:54+00:00,82


In [50]:
filename = '../data/interim/bitcoin_all_tweets_sentiments_20211001-20220130.csv'
file_exists = os.path.isfile(filename)

#Open/create a file to append data to
#csvFile = open(filename, 'a', newline='', encoding='utf8')

#Use csv writer
#csvWriter = csv.writer(csvFile)
if not file_exists:
    sData.to_csv(filename, mode='w')
sData.to_csv(filename, mode='a', header=False)
#csvFile.close()